In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
train_instance_num, pixel_num = mnist.train.images.shape
test_instance_num, class_num = mnist.test.labels.shape

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [5]:
x = tf.placeholder('float', shape=[None, pixel_num], name='x_images')
y_answer = tf.placeholder('float', shape=[None, class_num], name='y_labels')

In [6]:
x_image = tf.reshape(x, [-1, 28, 28, 1]) # 입력데이터를 원래 이미지의 구조로 재구성한다.
x_image

<tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>

In [7]:
conv1 = tf.layers.conv2d(inputs=x_image,
                filters=32,
                kernel_size=[3,3],
                padding='same',
                activation=tf.nn.relu,
                trainable=False,
                name='conv1')
conv1

<tf.Tensor 'conv1/Relu:0' shape=(?, 28, 28, 32) dtype=float32>

In [8]:
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2)
pool1

<tf.Tensor 'max_pooling2d/MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>

In [9]:
conv2 = tf.layers.conv2d(inputs=pool1, 
                         filters=64, 
                         kernel_size=[3,3], 
                         padding='same', 
                         activation=tf.nn.relu, 
                         trainable=True, 
                         name='conv2')
conv2

<tf.Tensor 'conv2/Relu:0' shape=(?, 14, 14, 64) dtype=float32>

In [10]:
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)
pool2

<tf.Tensor 'max_pooling2d_2/MaxPool:0' shape=(?, 7, 7, 64) dtype=float32>

In [11]:
pool2_flat = tf.reshape(pool2, [-1, pool2.shape[1].value*pool2.shape[2].value*pool2.shape[3].value])
pool2_flat

<tf.Tensor 'Reshape_1:0' shape=(?, 3136) dtype=float32>

In [12]:
dense1 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
dense1

<tf.Tensor 'dense/Relu:0' shape=(?, 1024) dtype=float32>

In [13]:
dense2 = tf.layers.dense(inputs=dense1, units=10, activation=tf.nn.softmax)
dense2

<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>

In [14]:
cross_entropy = - tf.reduce_sum(y_answer*tf.log(dense2)) # 크로스 엔트로피 방식으로 오차값을 선정
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(dense2, 1), tf.argmax(y_answer, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [15]:
saver = tf.train.Saver()
saver

In [17]:
saver.restore(sess, 'mnist_ckpt/total-layers-model')

INFO:tensorflow:Restoring parameters from mnist_ckpt/total-layers-model


In [24]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

['init',
 'x_images',
 'y_labels',
 'Reshape/shape',
 'Reshape',
 'conv1/kernel/Initializer/random_uniform/shape',
 'conv1/kernel/Initializer/random_uniform/min',
 'conv1/kernel/Initializer/random_uniform/max',
 'conv1/kernel/Initializer/random_uniform/RandomUniform',
 'conv1/kernel/Initializer/random_uniform/sub',
 'conv1/kernel/Initializer/random_uniform/mul',
 'conv1/kernel/Initializer/random_uniform',
 'conv1/kernel',
 'conv1/kernel/Assign',
 'conv1/kernel/read',
 'conv1/bias/Initializer/Const',
 'conv1/bias',
 'conv1/bias/Assign',
 'conv1/bias/read',
 'conv1/convolution/Shape',
 'conv1/convolution/dilation_rate',
 'conv1/convolution',
 'conv1/BiasAdd',
 'conv1/Relu',
 'max_pooling2d/MaxPool',
 'conv2/kernel/Initializer/random_uniform/shape',
 'conv2/kernel/Initializer/random_uniform/min',
 'conv2/kernel/Initializer/random_uniform/max',
 'conv2/kernel/Initializer/random_uniform/RandomUniform',
 'conv2/kernel/Initializer/random_uniform/sub',
 'conv2/kernel/Initializer/random_uniform

In [34]:
conv1_kernel = sess.graph.get_tensor_by_name('conv1/kernel:0')[0][0][0][0:10]
conv2_kernel = sess.graph.get_tensor_by_name('conv2/kernel:0')[0][0][0][0:10]
print(sess.run(conv1_kernel))
print(sess.run(conv2_kernel))

[-0.04327706  0.10378996  0.15576044 -0.10679744  0.03725898  0.02170954
 -0.15367696  0.15149818 -0.07106207  0.07409309]
[-0.05084438  0.00514666 -0.05015267  0.06133039 -0.00184576  0.02939141
  0.06201126 -0.06127528 -0.05732324  0.02453694]


In [35]:
tf.trainable_variables()

[<tf.Variable 'conv2/kernel:0' shape=(3, 3, 32, 64) dtype=float32_ref>,
 <tf.Variable 'conv2/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(3136, 1024) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(1024, 10) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32_ref>]

In [36]:
for i in range(1000):
    batch = mnist.train.next_batch(100) # 100 개씩 랜덤하게 픽해서 가져와 학습
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_answer:batch[1]})
        print("step %d, training accuracy : %g" % (i, train_accuracy))
        print(sess.run(conv1_kernel))
        print(sess.run(conv2_kernel))
        saver.save(sess, 'mnist_ckpt/part-layers-model', global_step=i, write_meta_graph=False)
    sess.run(train_step, feed_dict={x:batch[0], y_answer:batch[1]})

print("===최종학습 결과===")
print("test accuracy : %g" % sess.run(accuracy, feed_dict={x:mnist.test.images, y_answer:mnist.test.labels}))
saver.save(sess, 'mnist_ckpt/part-layers-model')

step 0, training accuracy : 0.97
[-0.04327706  0.10378996  0.15576044 -0.10679744  0.03725898  0.02170954
 -0.15367696  0.15149818 -0.07106207  0.07409309]
[-0.05084438  0.00514666 -0.05015267  0.06133039 -0.00184576  0.02939141
  0.06201126 -0.06127528 -0.05732324  0.02453694]
step 100, training accuracy : 0.99
[-0.04327706  0.10378996  0.15576044 -0.10679744  0.03725898  0.02170954
 -0.15367696  0.15149818 -0.07106207  0.07409309]
[-0.0509394   0.00455901 -0.04979987  0.06091654 -0.00128749  0.02936536
  0.06205282 -0.06180337 -0.05735058  0.02536311]
step 200, training accuracy : 0.98
[-0.04327706  0.10378996  0.15576044 -0.10679744  0.03725898  0.02170954
 -0.15367696  0.15149818 -0.07106207  0.07409309]
[-0.05068157  0.00436091 -0.05033682  0.06034876 -0.00168623  0.02882719
  0.06253471 -0.06262124 -0.05709907  0.02539417]
step 300, training accuracy : 0.97
[-0.04327706  0.10378996  0.15576044 -0.10679744  0.03725898  0.02170954
 -0.15367696  0.15149818 -0.07106207  0.07409309]
[

'mnist_ckpt/part-layers-model'